# TFM.

# Arbol de decisión basado en Random Forest

### Secciones

- Algoritmo Random Forest

In [8]:
import pyodbc

server = 'BIOE-MAJC' 
database = 'bioe' 
username = 'bioe' 
password = 'oe@LoCaL' 
trusted = 'yes;' 

cnxn = pyodbc.connect('Driver={SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=no;'+username+';PWD='+ password)
cursor = cnxn.cursor()

import pandas as pd


In [9]:
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
 
#cargamos los datos de entrada
query = "SELECT dG_S,benchmark, CP_BFKV,CP_BL,CP_BT,CP_GKS,CP_HLPL,CP_MJPL,CP_MJ3h,CP_MJ2h,CP_MJ1,CP_MJ2,CP_MSBM,CP_MS,CP_Qa,CP_Qm,CP_Qp" + \
  ",CP_RO,CP_SKOb,CP_SKOa,CP_SJKG,CP_TD,CP_TEl,CP_TEs,CP_TS,CP_VD,CP_SKOIP,AP_DCOMPLEX,AP_dDFIRE,AP_DFIRE2,CP_RMFCEN1" + \
  ",CP_RMFCEN2,CP_RMFCA,CP_TB,CP_TSC,AP_T1,AP_T2,AP_DOPE,AP_DOPE_HR,AP_ACE,INSIDE,HBOND,PI_PI,CAT_PI,ALIPH,ZRANK,ZRANK2" + \
  ",ROT_S,TRANS_S,NIPacking,NSC,FA_ATR,FA_REP,LK_SOLV,FA_PP,CG_VDW,CG_PP,CG_ENV,CG_BETA,HBOND2,AA_PROP,ROSETTADOCK,NHB" + \
  ",ELE,DESOLV,VDW,PYDOCK_TOT,ODA,PROPNSTS,SIPPER,AP_OPUS_PSP,AP_GEOMETRIC,AP_DARS,AP_URS,AP_MPS,AP_W1,CP_D1,AP_calRW" + \
  ",AP_calRWp,AP_PISA,FIREDOCK,FIREDOCK_AB,FIREDOCK_EI,CP_PIE,CP_DDG_U,CP_DDG_W,AP_DDG_U,AP_DDG_W,DDG_V FROM [bioe].[TFM].[tfm_NNaN0_S] ();"

df = pd.read_sql_query(query, cnxn)
print(df.head(5))
df.describe()
print(df.groupby("dG_S").size())

X_train, y_train = df.loc[df["benchmark"] == 1][["CP_BFKV","CP_BL","CP_BT","CP_GKS","CP_HLPL","CP_MJPL","CP_MJ3h","CP_MJ2h","CP_MJ1","CP_MJ2","CP_MSBM","CP_MS","CP_Qa","CP_Qm","CP_Qp" \
  ,"CP_RO","CP_SKOb","CP_SKOa","CP_SJKG","CP_TD","CP_TEl","CP_TEs","CP_TS","CP_VD","CP_SKOIP","AP_DCOMPLEX","AP_dDFIRE","AP_DFIRE2","CP_RMFCEN1" \
  ,"CP_RMFCEN2","CP_RMFCA","CP_TB","CP_TSC","AP_T1","AP_T2","AP_DOPE","AP_DOPE_HR","AP_ACE","INSIDE","HBOND","PI_PI","CAT_PI","ALIPH","ZRANK","ZRANK2" \
  ,"ROT_S","TRANS_S","NIPacking","NSC","FA_ATR","FA_REP","LK_SOLV","FA_PP","CG_VDW","CG_PP","CG_ENV","CG_BETA","HBOND2","AA_PROP","ROSETTADOCK","NHB" \
  ,"ELE","DESOLV","VDW","PYDOCK_TOT","ODA","PROPNSTS","SIPPER","AP_OPUS_PSP","AP_GEOMETRIC","AP_DARS","AP_URS","AP_MPS","AP_W1","CP_D1","AP_calRW" \
  ,"AP_calRWp","AP_PISA","FIREDOCK","FIREDOCK_AB","FIREDOCK_EI","CP_PIE","CP_DDG_U","CP_DDG_W","AP_DDG_U","AP_DDG_W","DDG_V"]], df.loc[df["benchmark"] == 1]["dG_S"]
X_test, y_test = df.loc[df["benchmark"] == 2][["CP_BFKV","CP_BL","CP_BT","CP_GKS","CP_HLPL","CP_MJPL","CP_MJ3h","CP_MJ2h","CP_MJ1","CP_MJ2","CP_MSBM","CP_MS","CP_Qa","CP_Qm","CP_Qp" \
  ,"CP_RO","CP_SKOb","CP_SKOa","CP_SJKG","CP_TD","CP_TEl","CP_TEs","CP_TS","CP_VD","CP_SKOIP","AP_DCOMPLEX","AP_dDFIRE","AP_DFIRE2","CP_RMFCEN1" \
  ,"CP_RMFCEN2","CP_RMFCA","CP_TB","CP_TSC","AP_T1","AP_T2","AP_DOPE","AP_DOPE_HR","AP_ACE","INSIDE","HBOND","PI_PI","CAT_PI","ALIPH","ZRANK","ZRANK2" \
  ,"ROT_S","TRANS_S","NIPacking","NSC","FA_ATR","FA_REP","LK_SOLV","FA_PP","CG_VDW","CG_PP","CG_ENV","CG_BETA","HBOND2","AA_PROP","ROSETTADOCK","NHB" \
  ,"ELE","DESOLV","VDW","PYDOCK_TOT","ODA","PROPNSTS","SIPPER","AP_OPUS_PSP","AP_GEOMETRIC","AP_DARS","AP_URS","AP_MPS","AP_W1","CP_D1","AP_calRW" \
  ,"AP_calRWp","AP_PISA","FIREDOCK","FIREDOCK_AB","FIREDOCK_EI","CP_PIE","CP_DDG_U","CP_DDG_W","AP_DDG_U","AP_DDG_W","DDG_V"]], df.loc[df["benchmark"] == 2]["dG_S"]

#print(X_train)
#print(y_train)

#print(X_test)
#print(y_test)


   dG_S  benchmark  CP_BFKV  CP_BL  CP_BT  CP_GKS  CP_HLPL  CP_MJPL  CP_MJ3h  \
0     1          1  -0.8654 -6.506  -5.48    -8.0    -28.6   -140.8    -2.96   
1     1          1  -2.3756 -6.130  -7.74    -2.4    -34.0   -178.0   -11.50   
2     1          1  -1.0114 -8.650  -4.96   -14.0    -34.2   -196.2     2.36   
3     0          1   1.1344 -2.076  -3.98    -4.0    -12.0   -122.4    -2.90   
4     0          1   1.5450 -4.558   2.74   -10.0    -26.8   -191.2     4.64   

   CP_MJ2h  ...  AP_PISA  FIREDOCK  FIREDOCK_AB  FIREDOCK_EI    CP_PIE  \
0  -184.80  ...  -0.2445    -58.02       -71.00       -25.76  1.138510   
1  -232.92  ...  -0.4878    -88.88       -96.94       -62.16  1.450095   
2  -158.48  ...  -0.0288    -36.54       -67.22        -3.33  1.475723   
3  -141.98  ...  -0.2250    -42.73       -47.67       -29.09  0.776228   
4  -155.10  ...  -0.2936    -45.19       -67.13        -4.81  1.071730   

    CP_DDG_U   CP_DDG_W   AP_DDG_U   AP_DDG_W      DDG_V  
0  -4.560318  -

C:\Users\coept\AppData\Local\Temp\ipykernel_11380\2975534309.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, cnxn)


#### Algoritmo Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=10, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, n_jobs=2,
                       random_state=1)

In [11]:
predictions = forest.predict(X_test)
print(accuracy_score(y_test, predictions))


0.5714285714285714


In [12]:
print(confusion_matrix(y_test, predictions))


[[ 4  8]
 [ 7 16]]


In [13]:
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.36      0.33      0.35        12
           1       0.67      0.70      0.68        23

    accuracy                           0.57        35
   macro avg       0.52      0.51      0.51        35
weighted avg       0.56      0.57      0.57        35

